In [1]:
import cv2
import torch
from torchvision.transforms.functional import to_tensor, to_pil_image
from torchvision.utils import draw_bounding_boxes
from ultralytics import YOLO
import numpy as np
import pyttsx3
import time

Initialize YOLOv8 model to use GPU

In [2]:
model = YOLO("yolov8n.pt")
model.overrides["device"] = "cuda"  # Force model to use GPU

Initialize text-to-speech engine

In [3]:
engine = pyttsx3.init()

In [4]:
voices = engine.getProperty('voices')

for voice in voices:
    print(f"Voice Name: {voice.name}, ID: {voice.id}")

Voice Name: Microsoft David Desktop - English (United States), ID: HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_DAVID_11.0
Voice Name: Microsoft Zira Desktop - English (United States), ID: HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0


In [5]:
engine.setProperty("rate", 150)  # Set speech rate
engine.setProperty("voice", "HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0")

Define the navigation system

In [6]:
def navigation_system():
    cap = cv2.VideoCapture(0)  # Access the camera
    if not cap.isOpened():
        print("Error: Camera not accessible")
        return

    # Variables for speaking navigation commands
    last_spoken_time = time.time()
    current_command = "Move Forward"  # Default command

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Failed to grab a frame")
                break

            # Convert frame to PyTorch tensor
            frame_tensor = to_tensor(frame).unsqueeze(0).to("cuda")  # Add batch dimension

            # Perform object detection using YOLOv8
            results = model(frame_tensor)
            detected_objects = results[0].boxes  # Access detected objects

            # Initialize variables for navigation
            navigation_command = "Move Forward"
            frame_height, frame_width, _ = frame.shape

            # Extract bounding boxes, labels, and confidences
            boxes = []
            labels = []
            confidence_threshold = 0.5
            for obj in detected_objects:
                if obj.conf >= confidence_threshold:
                    box = obj.xyxy[0].cpu()  # Bounding box coordinates
                    boxes.append(box)
                    labels.append(f"{model.names[int(obj.cls)]} {float(obj.conf):.2f}")

                    # Navigation logic
                    center_x = (box[0] + box[2]) / 2
                    center_y = (box[1] + box[3]) / 2

                    if frame_width * 0.3 <= center_x <= frame_width * 0.7:
                        navigation_command = "Stop"  # Object in the middle
                    elif center_x < frame_width * 0.3:
                        navigation_command = "Turn Left"
                    elif center_x > frame_width * 0.7:
                        navigation_command = "Turn Right"

            # Prepare frame for drawing
            frame_uint8 = torch.from_numpy(frame).permute(2, 0, 1).to(torch.uint8).cpu()  # Convert to CHW and uint8
            if boxes:
                frame_with_boxes = draw_bounding_boxes(
                    frame_uint8,
                    torch.stack(boxes).to(torch.int),
                    labels=labels,
                    colors="green",
                    width=2,
                )
                annotated_frame = to_pil_image(frame_with_boxes)
            else:
                annotated_frame = to_pil_image(frame_uint8)

            # Convert annotated frame back to RGB format for OpenCV
            annotated_frame = np.array(annotated_frame)

            # Overlay navigation command
            cv2.putText(
                annotated_frame,
                f"Command: {navigation_command}",
                (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 255, 0),
                2,
                cv2.LINE_AA,
            )

            # Speak the navigation command every 5 seconds
            current_time = time.time()
            if current_time - last_spoken_time >= 5:
                if current_command != navigation_command:
                    current_command = navigation_command
                engine.say(current_command)
                engine.runAndWait()
                last_spoken_time = current_time

            # Display the annotated frame
            cv2.imshow("Obstacle Avoidance System", annotated_frame)

            # Break loop on 'q' key press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    except Exception as e:
        print(f"Error during processing: {e}")

    finally:
        cap.release()
        cv2.destroyAllWindows()
        print("System shutdown.")

Run the navigation system

In [8]:
if __name__ == "__main__":
    navigation_system()


0: 480x640 1 person, 1 cat, 12.3ms
Speed: 0.0ms preprocess, 12.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 6.7ms
Speed: 0.0ms preprocess, 6.7ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 9.6ms
Speed: 0.0ms preprocess, 9.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 10.1ms
Speed: 0.0ms preprocess, 10.1ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.2ms
Speed: 0.0ms preprocess, 8.2ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.3ms
Speed: 0.0ms preprocess, 8.3ms inference, 3.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 9.5ms
Speed: 0.0ms preprocess, 9.5ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 9.1ms
Speed: 0.0ms preprocess, 9.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 64